In [0]:
%sql
USE CATALOG formula1;

In [0]:
dbutils.fs.ls('/Volumes/formula1/landing/landing_zone')

In [0]:
database = 'formula1'
schema = 'bronze'
landing_zone_path = '/Volumes/formula1/landing/landing_zone'

In [0]:
from pyspark.sql.functions import current_timestamp

for file in dbutils.fs.ls('dbfs:/Volumes/formula1/landing/landing_zone'):
    file_path = file[0]
    table_name = file[1].replace('.csv', '')

    df = (
        spark
        .read
        .option('header', True)
        .option('inferSchema', True)
        .csv(f'{file_path}')
    )

    df = df.withColumn('bronze_ingestion', current_timestamp())

    (df.write
     .format('delta')
     .mode('overwrite')
     .option('mergeSchema', True)
     .saveAsTable(f'{database}.{schema}.{table_name}'))

    
